## Part 4: STM images

Assistants:Xing Wang & Moloud Kaviani

Room:	N216

Phone:	031 631 56 25

Email:	xing.wang@dcb.unibe.ch, moloud.kaviani@dcb.unibe.ch  

## 1. Introduction
In the following we want to compute STM images of the adsorbed oxygen species. It is however instructive to first compute those for a more covalent surface to learn how to interpret the results. We will hence compute STM images for graphite and graphene to see the differences in bonding in the two structures.

The general procedure to compute an STM image is the following:

* Compute a self-consistent electronic structure
* Compute the density of states integrated between the Fermi energy and the bias voltage as given by equation.
* Conversion of this 3D density dataset into a constant-current dataset.
* Visualization of the results.

### 2. STM image of graphite surface
First we build the surface of graphite. 
Graphite consists of sequences of stacked graphene sheets as shown in Fig. 1.
<img src="images/graphite.png"  width="500">

Figure 1: Atomic geometry of graphite. The structure is made up of layers of graphene sheets. There are grey ($\alpha$) and red ($\beta$) carbon atoms, that are different in the sense that $\alpha$ carbon atoms have another carbon directly underneath whereas $\beta$ carbon atoms do not. This difference hence comes from the graphite structure and will not exist for a single graphene sheet.

In [ ]:
from ase.lattice.hexagonal import Graphite
from ase.build import surface
from x3dase.visualize import view_x3d_n
# build graphite bulk, we read the lattice constant from Figure 1. a = 246pm, c = 334.8*2pm
graphite = Graphite('C', latticeconstant={'a':2.460, 'c': 6.696},size=(1,1,1))
# build graphite surface
graphite = surface(graphite, (0, 0, 1), 2, vacuum = 8.0)
view_x3d_n(graphite*[3, 3, 1], bond=1.0, label = True, output = 'htmls/graphite.html')

Then calculate the energy of the graphite surface.

In [ ]:
from xespresso import Espresso

pseudopotentials = {'C': 'C.pbe-n-rrkjus_psl.1.0.0.UPF',}
queue = {'ntasks': 20, 'partition': 'empi', 'time': '00:59:00'}
calc = Espresso(pseudopotentials = pseudopotentials, 
                label  = 'surface/graphite',
                ecutwfc = 40,
                occupations = 'smearing',
                degauss = 0.01,
                vdw_coor = 'grimme-d2',
                kpts=(20, 20, 1),
                queue = queue,
               )
graphite.calc = calc
e = graphite.get_potential_energy()
print('Energy: {0:1.3f}'.format(e))

Now let's run a post calculation using package `pp`. This will produce the stm_1.0eV.cube and stm_0.1eV.cube file, which contains on a 3D grid all the values of the DOS integrated between E$_{fermi}$ and E$_{fermi+eV_{bias}}$.

Please look at the comment lines for the meaning of the paramters

In [ ]:
from ase.io.cube import read_cube_data
import pickle

queue = {'ntasks': 20, 'partition': 'debug', 'time': '00:10:00'}
for bias in [0.1, 1.0]:
    calc.post(queue = queue, 
            package='pp', 
            plot_num = 5,               # Compute a STM dataset
            sample_bias = bias/13.6057, # The applied bias voltage in Ry (1 Ry = 13.6057 eV)
            iflag = 3,                  # Output a 3D dataset of integrated local DOS
            output_format = 6,          # Output as an cube file
            fileout = 'stm_{0}eV.cube'.format(bias)   # Name of the output file
            )
    # read the cube file and save it to 'datas.pickle', which will be used by the STM method in ASE
    ldos, atoms = read_cube_data('surface/graphite/stm_{0}eV.cube'.format(bias))
    with open('surface/graphite/stm_{0}.pickle'.format(bias), 'wb') as f:
        pickle.dump([ldos, bias, atoms.cell], f)

Now let's initialize a STM object, and make a constant height scan at $z$=19.5 Å (for our surface, the top layer is at $z$=18.04 Å) and plot it. 

Change the $z$ value to see how the image changes as a function of the $z$-coordinate (the sample-tip separation).

Also try different bias values (0.1 and 1.0).

In [ ]:
from ase.dft.stm import STM
from ase.io.cube import read_cube_data
import pickle
import matplotlib.pyplot as plt

bias = 0.1
z = 19.0
stm = STM('surface/graphite/stm_{0}.pickle'.format(bias))
x, y, I = stm.scan2(bias, z, repeat=(3, 3))

plt.figure()
plt.gca(aspect='equal')
plt.contourf(x, y, I, 40, cmap=plt.cm.bone)
plt.colorbar()
plt.savefig('images/STM-2d-I-graphite-bias-{0}-z-{1}.png'.format(bias, z))

We had seen in the introduction that experimental STM images are usually recorded in “constant-current” mode. Now get the averaged current at $z$=19.5 Å. From the current we make a scan to get a 2-d array of constant current scan and make a contour plot.

In [ ]:
bias = 0.1
z = 19.5
c = stm.get_averaged_current(bias, z)
x, y, h = stm.scan(bias, c, repeat=(3, 3))
h[h<19.0] = 19.0
plt.gca(aspect='equal')
plt.contourf(x, y, h, 40, cmap=plt.cm.bone, )
plt.colorbar()
plt.savefig('images/STM-2d-H-graphite-bias-{0}-z-{1}.png'.format(bias, z))


Do you understand what you see? If not, let’s try to learn some more about the electronic structure to help us interpret the images. For this we will compute electronic band structures – think of them as the equivalent of molecular orbital diagrams in periodic systems.

In graphite, the carbon atoms form an sp$^2$ bonded hexagonal lattice. In this configuration the s, p$_x$ and p$_y$-derived σ orbitals are occupied by 3 electrons and the p$_z$-derived π orbital contains 1 electron. The Fermi energy lies in this π band and for small bias voltages we will primarily see the DOS coming from these p$_z$-derived orbitals.

In order to check exactly how small these bias voltages can be, we will compute the band-structures for graphite.

### 3. Band structure for graphite bulk
We first run a normal calculation, and get the fermi energy using

`fermi = calc.get_fermi_level()` 

In [ ]:
from ase.lattice.hexagonal import Graphite
from xespresso import Espresso

graphite = Graphite('C', latticeconstant={'a':2.460, 'c': 6.696},size=(1,1,1))
pseudopotentials = {'C': 'C.pbe-n-rrkjus_psl.1.0.0.UPF',}
queue = {'ntasks': 20, 'partition': 'debug', 'time': '00:10:00'}
calc = Espresso(pseudopotentials = pseudopotentials, 
                label  = 'bulk/graphite',
                ecutwfc = 40,
                occupations = 'smearing',
                degauss = 0.01,
                vdw_coor = 'grimme-d2',
                kpts=(10, 10, 4),
                queue = queue,
               )
graphite.calc = calc
e = graphite.get_potential_energy()
fermi = graphite.calc.get_fermi_level()
print('Energy: {0:1.3f}'.format(e))
print('Fermie energy: {0:1.3f}'.format(fermi))


In a second step, we want to calculate the eigenvalues of the kinetic energy operator along given lines in $k$-space to obtain the electronic band structure. For this, we do a non self-consistent calculation using `nscf` function, and set `calculation = 'bands'` and modify the $k$-point to a $k$-point path.
`kpts={'path': 'GMKG', 'npoints': 50}`. 

This will tell Quantum ESPRESSO to calculate a string of lines in $k$-space going from (0, 0, 0) to (0.5, 0, 0) to ($\frac{1}{3}$, $\frac{1}{3}$, 0) and back to (0, 0, 0) making 50 points along each segment. In Figure 2, this corresponds to moving from $Γ$ to $M$ to $K$ and back to $Γ$. We have left the k$_z$ direction out on purpose as later we want to compare to graphene and the $z$ direction is aperiodic for graphene, giving it a 2D reciprocal space.

<img src="images/bz-graphite.png"  width="500">

Figure 2: Reciprocal space (the first Brillouin zone actually) for graphite. The points of interest for us here are $Γ$, $M$ and $K$ at coordinates (0, 0, 0), ($\frac{1}{2}$, 0, 0) and ($\frac{1}{3}$, $\frac{1}{3}$, 0) respectively.

In [ ]:
import matplotlib.pyplot as plt

calc = graphite.calc
kpts={'path': 'GMKG', 'npoints': 50}
calc.nscf(queue = queue, calculation = 'bands', kpts = kpts)
calc.nscf_calculate()
calc.read_results()
bs = calc.band_structure()
bs._reference = fermi
bs = bs.subtract_reference()
ax = bs.plot(emin = -15, emax=10)
plt.savefig('images/graphite-band.png')


Can you based on this figure understand why the graphite STM images at 0.1 eV and 1.0 eV look this different? Discuss this with your assistant if you are unsure.

### 4. Computing STM images of graphene
We will now exfoliate graphite to make a graphene sheet. Experimentally you would need scotch tape and patience but computationally it’s a lot easier. 



In [ ]:
from ase.io import read
from ase.build import surface
from x3dase.visualize import view_x3d_n

graphite_bulk = Graphite('C', latticeconstant={'a':2.460, 'c': 6.696},size=(1,1,1))
# build graphite surface, which contains two layers of carbon atoms
graphene = surface(graphite_bulk, (0, 0, 1), 1, vacuum = 5.0)
view_x3d_n(graphene*[3, 3, 1], bond=1.0, label = True, output = 'htmls/graphene-2.html')

In [ ]:
# select all atoms in the top layer.
# To do that, please find the z coordinate of the atoms, and put a value a little bit smaller than the coordiante in the #todo.
select = [atom.index for atom in graphene if atom.z > 8]
# delete the select atoms
del graphene[select]
view_x3d_n(graphene*[3, 3, 1], bond=1.0, label = True, output = 'htmls/graphene.html')


Now run this calculation and proceed as before to compute a band structure and to calculate STM images at biases of 1.0 and 0.1 eV. Compare the band structures and STM images for graphite with the ones for graphene. Notice that they are similar but that the one for graphite has additional features. Think about the origin of these features in terms of alpha and beta carbon atoms shown in Figure 1.

In [ ]:
from xespresso import Espresso
from ase.io.cube import read_cube_data
import pickle

pseudopotentials = {'C': 'C.pbe-n-rrkjus_psl.1.0.0.UPF',}
queue = {'ntasks': 20, 'partition': 'empi', 'time': '00:59:00'}
calc = Espresso(pseudopotentials = pseudopotentials, 
                label  = 'surface/graphene',
                ecutwfc = 40,
                occupations = 'smearing',
                degauss = 0.01,
                vdw_coor = 'grimme-d2',
                kpts=(20, 20, 1),
                queue = queue,
               )
graphene.calc = calc
e = graphene.get_potential_energy()
fermi = graphene.calc.get_fermi_level()
print('Energy: {0:1.3f}'.format(e))
print('Fermie energy: {0:1.3f}'.format(fermi))

In [ ]:
import numpy as np
calc = graphene.calc
for bias in [0.1, 1.0]:
    calc.post(queue = queue, 
            package='pp', 
            plot_num = 5,               # Compute a STM dataset
            sample_bias = bias/13.6057, # The applied bias voltage in Ry (1 Ry = 13.6057 eV)
            iflag = 3,                  # Output a 3D dataset of integrated local DOS
            output_format = 6,          # Output as an cube file
            fileout = 'stm_{0}eV.cube'.format(bias)   # Name of the output file
            )
    # read the cube file and save it to 'datas.pickle', which will be used by the STM method in ASE
    ldos, atoms = read_cube_data('surface/graphene/stm_{0}eV.cube'.format(bias))
    with open('surface/graphene/stm_{0}.pickle'.format(bias), 'wb') as f:
        pickle.dump([ldos, bias, atoms.cell], f)

In [ ]:
from ase.dft.stm import STM
from ase.io.cube import read_cube_data
import pickle
import matplotlib.pyplot as plt

bias = 1.0
z = 7
stm = STM('surface/graphene/stm_{0}.pickle'.format(bias))
x, y, I = stm.scan2(bias, z, repeat=(3, 3))

plt.figure()
plt.gca(aspect='equal')
plt.contourf(x, y, I, 40, cmap=plt.cm.bone)
plt.colorbar()
plt.savefig('images/STM-2d-I-graphene-bias-{0}-z-{1}.png'.format(bias, z))

In [ ]:
z = 7
c = stm.get_averaged_current(bias, z)
x, y, h = stm.scan(bias, c, repeat=(3, 3))
plt.gca(aspect='equal')
plt.contourf(x, y, h, 40, cmap=plt.cm.bone, )
plt.colorbar()
plt.savefig('images/STM-2d-H-graphene-bias-{0}-z-{1}.png'.format(bias, z))


### 4.2 Band structure of graphene
Similar to graphite, here we calculate the band structure of graphene.

In [ ]:
import matplotlib.pyplot as plt

calc = graphene.calc
kpts={'path': 'GMKG', 'npoints': 50}
calc.nscf(queue = queue, calculation = 'bands', kpts = kpts)
calc.nscf_calculate()
calc.read_results()
bs = calc.band_structure()
bs._reference = fermi
bs = bs.subtract_reference()
ax = bs.plot(emin = -20, emax=10)
plt.savefig('images/graphene-band.png')


From the band structures of graphite (bulk) and graphene, we get a pretty good idea which bands we will see at a given bias voltage in graphite and graphene. Try to identify the bands as σ or π bands. Discuss profusely with your assistant for this step. 

### 5. STM Images of O adsorbates on Pt (111)
Now go back to your already computed Pt (111) surfaces with adsorbed molecular and atomic oxygen and compute STM images with bias voltages of 0.1 eV, 0.2 eV, 0.5 eV and 1.0 eV. In the figure below, very clever researchers have used voltage pulses of the STM tip (Figure b) to successively manipulate adsorbed O$_2$ molecules. Compare your images with these and try to see if you can match the computed STM images with the experimental ones.
<img src="images/stm-pt-o.png"  width="500">

Figure 3: STM images of tip-manipulated O$_2$ molecules on a Pt (111) surface. A) shows two O$_2$ molecules adsorbed on fcc sites. Then a voltage pulse (b) is applied with the tip over the right molecule, which leads to its dissociation with one O at a hcp site and one at the fcc site. A second pulse on the left molecule then leads to two additional O at hcp sites. Figure taken from Stipe et al., Phys. Rev. Lett., 78, 4410-4413, (1997).

#### 5.1  Dissociative adsorption
Here, we calculate the two most stable sites (fcc and hcp) for oxygen atoms. We check the structures from our preivous calculations (Part 3.2).

In [ ]:
site = 'fcc'
calc = Espresso(label = 'surface/pt-o-{0}'.format(site))
calc.read_results()
atoms = calc.results['atoms']
view_x3d_n(atoms, bond = 1.0, output = 'htmls/pt-o-{0}-relax.html'.format(site))

In [ ]:
# we only need read data from previous calculation.
for site in ['fcc', 'hcp']:
    calc = Espresso(label = 'surface/pt-o-{0}'.format(site))
#     for bias in [0.1]:
    for bias in [0.1, 0.2, 0.5, 1.0]:
        calc.post(queue = queue, 
            package='pp', 
            plot_num = 5,               # Compute a STM dataset
            sample_bias = bias/13.6057, # The applied bias voltage in Ry (1 Ry = 13.6057 eV)
            iflag = 3,                  # Output a 3D dataset of integrated local DOS
            output_format = 6,          # Output as an cube file
            fileout = 'stm_{0}eV.cube'.format(bias)   # Name of the output file
            )
        # read the cube file and save it to 'datas.pickle', which will be used by the STM method in ASE
        ldos, atoms = read_cube_data('surface/pt-o-{0}/stm_{1}eV.cube'.format(site, bias))
        with open('surface/pt-o-{0}/stm_{1}.pickle'.format(site, bias), 'wb') as f:
            pickle.dump([ldos, bias, atoms.cell], f)



Then we calculate the STM images in “constant-current” mode, and compare with experimental images.

In [ ]:
from ase.dft.stm import STM
from ase.io.cube import read_cube_data
import pickle
import matplotlib.pyplot as plt

z = 13.0

for site in ['fcc', 'hcp']:
    for bias in [0.1, 0.5, 1.0]:
        stm = STM('surface/pt-o-{0}/stm_{1}.pickle'.format(site, bias))
        c = stm.get_averaged_current(bias, z)
#         c = 0.0001
        x, y, h = stm.scan(bias, c, repeat=(3, 3))
        h[h<12.5] = 12.5
        plt.figure()
        plt.gca(aspect='equal')
        plt.contourf(x, y, h, 40, cmap=plt.cm.bone, )
        plt.colorbar()
        plt.savefig('images/STM-2d-H-pt-o-{0}-bias-{1}-z-{2}.png'.format(site, bias, z))


#### 5.2 Molecular adsorption
Here, we calculate the two most stable sites (top-top and top-fcc) for oxygen molecular adsorption. We check the structures from our preivous calculations (Part 3.3).

In [ ]:
site = 'top-top'
calc = Espresso(label = 'surface/pt-o2-{0}'.format(site))
calc.read_results()
atoms = calc.results['atoms']
view_x3d_n(atoms, bond = 1.0, output = 'htmls/pt-o2-{0}-relax.html'.format(site))

In [ ]:
# we only need read data from previous calculation.
for site in ['top-top', 'top-fcc']:
    calc = Espresso(label = 'surface/pt-o2-{0}'.format(site))
#     for bias in [0.1]:
    for bias in [0.1, 0.2, 0.5, 1.0]:
        calc.post(queue = queue, 
            package='pp', 
            plot_num = 5,               # Compute a STM dataset
            sample_bias = bias/13.6057, # The applied bias voltage in Ry (1 Ry = 13.6057 eV)
            iflag = 3,                  # Output a 3D dataset of integrated local DOS
            output_format = 6,          # Output as an cube file
            fileout = 'stm_{0}eV.cube'.format(bias)   # Name of the output file
            )
        # read the cube file and save it to 'datas.pickle', which will be used by the STM method in ASE
        ldos, atoms = read_cube_data('surface/pt-o2-{0}/stm_{1}eV.cube'.format(site, bias))
        with open('surface/pt-o2-{0}/stm_{1}.pickle'.format(site, bias), 'wb') as f:
            pickle.dump([ldos, bias, atoms.cell], f)



Then we calculate the STM images in “constant-current” mode, and compare with experimental images.

In [ ]:
from ase.dft.stm import STM
from ase.io.cube import read_cube_data
import pickle
import matplotlib.pyplot as plt

z = 13.0

for site in ['top-top', 'top-fcc']:
    for bias in [0.1, 0.5, 1.0]:
        stm = STM('surface/pt-o2-{0}/stm_{1}.pickle'.format(site, bias))
        c = stm.get_averaged_current(bias, z)
#         c = 0.0001
        x, y, h = stm.scan(bias, c, repeat=(3, 3))
        h[h<12.5] = 12.5
        plt.figure()
        plt.gca(aspect='equal')
        plt.contourf(x, y, h, 40, cmap=plt.cm.bone, )
        plt.colorbar()
        plt.savefig('images/STM-2d-H-pt-o2-{0}-bias-{1}-z-{2}.png'.format(site, bias, z))


### 6. Conclusion
Comment on the usefulness of density functional calculations in interpreting surface science results. What can we test in a computer that experiment cannot easily do? Again, discuss this
with your assistant.